In [1]:
!pip install xgboost

In [2]:
%load_ext autoreload
%autoreload 1
%aimport transformations

transformations loaded, version: 0.1


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython.display import clear_output

import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 999
pd.options.display.max_rows = 113

import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.preprocessing import StandardScaler, PowerTransformer

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture

RS = 35577 # global random state seed
data_path = './data-raw/'

%aimport transformations
%aimport f

framework loaded, version: 0.1


In [4]:
(X, y) = f.from_pkl(f'data-processed/all_b1_b2.data.pkl')

X.shape
X.head(3)

y.shape
y.head(3)

(200, 36)

,x__amin,x__amax,x__sum,x__median,x__mean,x__std,x__var,x__q1,x__q3,x__iqr,x__kurtosis,x__skew,y__amin,y__amax,y__sum,y__median,y__mean,y__std,y__var,y__q1,y__q3,y__iqr,y__kurtosis,y__skew,z__amin,z__amax,z__sum,z__median,z__mean,z__std,z__var,z__q1,z__q3,z__iqr,z__kurtosis,z__skew
0,-6.716668,5.307868,-10988.070315,-0.079124,-0.134299,1.359987,1.849565,-0.944891,0.690446,1.635337,0.645580,-0.260035,-5.812267,5.638507,-2005.749440,-0.039563,-0.024515,1.090317,1.188792,-0.702004,0.622879,1.324883,0.845721,0.190106,-8.080924,8.244501,-15909.421691,-0.206307,-0.194449,1.503315,2.259956,-1.070594,0.754012,1.824606,0.873858,-0.226021
1,-12.103661,11.753020,-18844.323785,-0.175320,-0.239902,2.477716,6.139077,-1.522068,1.075231,2.597300,1.408079,-0.167536,-8.745936,12.452191,-318.869239,-0.134197,-0.004059,2.376120,5.645946,-1.459080,0.906782,2.365862,1.936212,0.887947,-10.193626,10.357203,-6543.554226,-0.110275,-0.083304,2.156482,4.650413,-1.262658,1.138140,2.400798,0.657898,0.065098
2,-7.678631,6.558420,-12546.141906,-0.079124,-0.136898,1.350739,1.824495,-0.752498,0.498054,1.250552,1.485456,-0.097227,-4.298115,5.259969,5780.428883,0.055072,0.063073,0.822131,0.675900,-0.323466,0.433610,0.757076,2.150199,0.258396,-10.289657,8.244501,12884.241287,0.177821,0.140587,1.480194,2.190974,-0.398371,0.850044,1.248415,3.049378,-0.710093


(200,)

0    1
1    1
2    1
Name: status, dtype: int64

# Models

https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

In [5]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

# CV

In [22]:
def cross_val(model_factory, X, y, cv_repeat_n = 1, model_repeat_n = 1, folds_n = 7):
    np.random.seed(RS)
    get_random = lambda  : np.random.randint(1, 10000)
    cv_scores = []
    
    for i_cv in range(cv_repeat_n):
        fold_scores = []
        cv_scores.append(fold_scores)
        
        for i_fold, (idx_train, idx_test) in enumerate(StratifiedKFold(n_splits=folds_n, shuffle=True, random_state=get_random()).split(X, y, y)):
            
            X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
            X_test, y_test = X.iloc[idx_test], y.iloc[idx_test]
            
            model_scores = []
            fold_scores.append(model_scores)

            for i_model in range(model_repeat_n):
                rand = get_random()
                constructor, params_dic = model_factory(rand)
                
                model = constructor(**params_dic)
                # model = model_factory(get_random())
                model.fit(X_train, y_train)

                y_pred = model.predict_proba(X_test)[:, 1]
                score = roc_auc_score(y_test, y_pred)
                model_scores.append(score)
                
            model_name = model.__class__.__name__
            model_params = params_dic.copy()
                
    return model_name, model_params, cv_repeat_n, folds_n, model_repeat_n, np.array(cv_scores)

In [23]:
def get_stats(results):
    idx = results[0]
    
    stats_ = results[5]
    return pd.DataFrame(
        {
            'params' : str(results[1]),
            'cv_repeat_n' : results[2],
            'folds_n' : results[3],    
            'model_repeat_n' : results[4],

            'min': stats_.min(),
            'mean': stats_.mean(),
            'max': stats_.max(),
            'std': stats_.std(),

            'fold_min': stats_.mean(axis=2).reshape(1,-1).min(),
            'fold_mean': stats_.mean(axis=2).reshape(1,-1).mean(),
            'fold_max': stats_.mean(axis=2).reshape(1,-1).max(),
            'fold_std': stats_.mean(axis=2).reshape(1,-1).std(),
        },
        index=[idx]
    )

## Multiple Models

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

# from sklearn.svm import SVC
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# from sklearn.naive_bayes import GaussianNB
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [29]:
all_models = [
    lambda random_state: (DecisionTreeClassifier, {}),
    lambda random_state: (RandomForestClassifier, {'random_state':random_state}),    
    lambda random_state: (AdaBoostClassifier,     {'random_state':random_state}),
    lambda random_state: (KNeighborsClassifier,   {}),
    lambda random_state: (LGBMClassifier,         {'random_state':random_state}),
    lambda random_state: (XGBClassifier,          {'random_state':random_state}),
]

In [30]:
%%time 

all_results = []

for factory in all_models:
    for cv_i in [1]:
        for fold_i in [7]:
            for model_i in [5]:
                results = cross_val(
                        factory,
                        X, y, 
                        cv_repeat_n = cv_i,
                        model_repeat_n = model_i,
                        folds_n = fold_i
                    )
                all_results.append(get_stats(results))
    clear_output(wait=True)
    display(pd.concat(all_results).round(3))

clear_output(wait=True)
pd.concat(all_results).round(3)

CPU times: user 3min 22s, sys: 1 s, total: 3min 23s
Wall time: 25.1 s


,params,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
DecisionTreeClassifier,{},1,7,5,0.643,0.858,0.967,0.087,0.664,0.858,0.924,0.084
RandomForestClassifier,{'random_state': 2404},1,7,5,0.944,0.985,1.000,0.018,0.951,0.985,1.000,0.018
AdaBoostClassifier,{'random_state': 2404},1,7,5,0.944,0.988,1.000,0.018,0.949,0.988,1.000,0.018
KNeighborsClassifier,{},1,7,5,0.883,0.951,0.990,0.043,0.883,0.951,0.990,0.043
LGBMClassifier,{'random_state': 2404},1,7,5,0.964,0.989,1.000,0.014,0.964,0.989,1.000,0.014
XGBClassifier,{'random_state': 2404},1,7,5,0.954,0.981,1.000,0.019,0.954,0.981,1.000,0.019


In [90]:
s = pd.concat(all_results)
s.style.bar(subset='min')

,params,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
DecisionTreeClassifier,{},1,7,5,0.642857,0.858095,0.966667,0.087020,0.664286,0.858095,0.924286,0.084017
RandomForestClassifier,{'random_state': 2404},1,7,5,0.943878,0.984592,1.000000,0.018496,0.950510,0.984592,1.000000,0.018097
AdaBoostClassifier,{'random_state': 2404},1,7,5,0.943878,0.987755,1.000000,0.017666,0.948980,0.987755,1.000000,0.017623
KNeighborsClassifier,{},1,7,5,0.882653,0.951215,0.990476,0.042844,0.882653,0.951215,0.990476,0.042844
LGBMClassifier,{'random_state': 2404},1,7,5,0.964286,0.989116,1.000000,0.014303,0.964286,0.989116,1.000000,0.014303
XGBClassifier,{'random_state': 2404},1,7,5,0.954082,0.981147,1.000000,0.019299,0.954082,0.981147,1.000000,0.019299


In [94]:
s['min'].max()

1.0

DecisionTreeClassifier    0.000000
RandomForestClassifier    0.312169
AdaBoostClassifier        0.312169
KNeighborsClassifier      0.248677
LGBMClassifier            0.333333
XGBClassifier             0.322751
Name: min, dtype: float64

In [104]:
s = pd.concat(all_results)
s['min'] = ( s['min'] - s['min'].min() ) / s['min'].max()
s.style.bar(subset='min', vmin=0, vmax=1)

,params,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
DecisionTreeClassifier,{},1,7,5,0.000000,0.858095,0.966667,0.087020,0.664286,0.858095,0.924286,0.084017
RandomForestClassifier,{'random_state': 2404},1,7,5,0.312169,0.984592,1.000000,0.018496,0.950510,0.984592,1.000000,0.018097
AdaBoostClassifier,{'random_state': 2404},1,7,5,0.312169,0.987755,1.000000,0.017666,0.948980,0.987755,1.000000,0.017623
KNeighborsClassifier,{},1,7,5,0.248677,0.951215,0.990476,0.042844,0.882653,0.951215,0.990476,0.042844
LGBMClassifier,{'random_state': 2404},1,7,5,0.333333,0.989116,1.000000,0.014303,0.964286,0.989116,1.000000,0.014303
XGBClassifier,{'random_state': 2404},1,7,5,0.322751,0.981147,1.000000,0.019299,0.954082,0.981147,1.000000,0.019299


In [105]:
s = pd.concat(all_results)
s.style.bar(subset='min', vmin=s['min'].min(), vmax=s['min'].max())

,params,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
DecisionTreeClassifier,{},1,7,5,0.642857,0.858095,0.966667,0.087020,0.664286,0.858095,0.924286,0.084017
RandomForestClassifier,{'random_state': 2404},1,7,5,0.943878,0.984592,1.000000,0.018496,0.950510,0.984592,1.000000,0.018097
AdaBoostClassifier,{'random_state': 2404},1,7,5,0.943878,0.987755,1.000000,0.017666,0.948980,0.987755,1.000000,0.017623
KNeighborsClassifier,{},1,7,5,0.882653,0.951215,0.990476,0.042844,0.882653,0.951215,0.990476,0.042844
LGBMClassifier,{'random_state': 2404},1,7,5,0.964286,0.989116,1.000000,0.014303,0.964286,0.989116,1.000000,0.014303
XGBClassifier,{'random_state': 2404},1,7,5,0.954082,0.981147,1.000000,0.019299,0.954082,0.981147,1.000000,0.019299


In [123]:
s.style.background_gradient(subset=['min', 'mean', 'max', 'std'], cmap='Blues')

,params,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
DecisionTreeClassifier,{},1,7,5,0.642857,0.858095,0.966667,0.087020,0.664286,0.858095,0.924286,0.084017
RandomForestClassifier,{'random_state': 2404},1,7,5,0.943878,0.984592,1.000000,0.018496,0.950510,0.984592,1.000000,0.018097
AdaBoostClassifier,{'random_state': 2404},1,7,5,0.943878,0.987755,1.000000,0.017666,0.948980,0.987755,1.000000,0.017623
KNeighborsClassifier,{},1,7,5,0.882653,0.951215,0.990476,0.042844,0.882653,0.951215,0.990476,0.042844
LGBMClassifier,{'random_state': 2404},1,7,5,0.964286,0.989116,1.000000,0.014303,0.964286,0.989116,1.000000,0.014303
XGBClassifier,{'random_state': 2404},1,7,5,0.954082,0.981147,1.000000,0.019299,0.954082,0.981147,1.000000,0.019299


In [74]:
s['min']/s['min'].min()*100

DecisionTreeClassifier    100.000000
RandomForestClassifier    146.825397
AdaBoostClassifier        146.825397
KNeighborsClassifier      137.301587
LGBMClassifier            150.000000
XGBClassifier             148.412698
Name: min, dtype: float64

In [112]:
s.style.background_gradient(subset='min')

,params,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,fold_min,fold_mean,fold_max,fold_std
DecisionTreeClassifier,{},1,7,5,0.642857,0.858095,0.966667,0.087020,0.664286,0.858095,0.924286,0.084017
RandomForestClassifier,{'random_state': 2404},1,7,5,0.943878,0.984592,1.000000,0.018496,0.950510,0.984592,1.000000,0.018097
AdaBoostClassifier,{'random_state': 2404},1,7,5,0.943878,0.987755,1.000000,0.017666,0.948980,0.987755,1.000000,0.017623
KNeighborsClassifier,{},1,7,5,0.882653,0.951215,0.990476,0.042844,0.882653,0.951215,0.990476,0.042844
LGBMClassifier,{'random_state': 2404},1,7,5,0.964286,0.989116,1.000000,0.014303,0.964286,0.989116,1.000000,0.014303
XGBClassifier,{'random_state': 2404},1,7,5,0.954082,0.981147,1.000000,0.019299,0.954082,0.981147,1.000000,0.019299


In [125]:
s.style.bar(

<bound method Styler.bar of <pandas.io.formats.style.Styler object at 0x7f38be2ab520>>